Social network analysis with NetworkX
=====

In the last lesson we learned about Graphviz, which is a well-known and mature package for  network graphs - that is, graphs made up of nodes that are linked with edges, and that may have extra information (metadata) as attributes on any of the graph, the nodes, or the edges.

Graphviz has one job and does it well: given a set of nodes and edges and given some attributes that say how they should look, it makes the best possible visual picture, or layout, of the graph. What Graphviz doesn't support is any sort of mathematical operation or analysis on the graph, beyond counting the nodes and edges - you can't say whether there is a loop, or whether there are unconnected nodes, or which nodes are the most connected, or how many distinct groups there are. For this we need other tools, and a popular one is NetworkX. You can find the documentation about NetworkX at [this link](https://networkx.github.io/documentation/networkx-1.10/overview.html).

In [ ]:
import networkx as nx

Like Graphviz, NetworkX understands nodes, edges, and attributes in a graph. Unlike Graphviz, NetworkX knows next to nothing about how best to draw a picture of a graph. Its purpose is, rather, to analyze a graph that is given. It's quite common therefore to make a graph using NetworkX, run some analysis, set some visual properties based on the results of the analysis, and then convert the graph to Graphviz for display.

This means that, at first sight, NetworkX works a lot like Graphviz does. You make a graph, and you add nodes and edges.

In [ ]:
undirected_graph = nx.Graph()
directed_graph = nx.DiGraph()

undirected_graph.add_edge("Alice", "Bob")
undirected_graph.add_edge("Bob", "Carol")

directed_graph.add_edge("Alice", "Bob")
directed_graph.add_edge("Bob", "Carol")

Since we have Graphviz installed, we can use it to print out the graphs as we did last week, though we need a slightly different Python module for this (`pygraphviz` rather than `graphviz`)!

In [ ]:
%load_ext hierarchymagic

In [ ]:
%dotstr -f svg nx.nx_agraph.to_agraph(undirected_graph).string()
%dotstr -f svg nx.nx_agraph.to_agraph(directed_graph).string()

But, of course, the reason we use NetworkX is to find things out about graphs. Let's try it out, with an arbitrary graph.

In [ ]:
# This gives us an undirected graph with 30 nodes, and each node has a 5% chance 
# of being connected to each other node.
random_graph = nx.erdos_renyi_graph(30, 0.05)  

# Now we can look at it.
%dotstr -f svg nx.nx_agraph.to_agraph(random_graph).string()

The graph generated will have some nodes that are connected together and some that aren't connected to anything. **Your graph will look different from mine!** NetworkX allows us to find these groups.

In [ ]:
for subgraph in nx.connected_components(random_graph):
    print(subgraph)

We can also find out which node has the most connections. The number of edges in and out of a node is called its **degree**. NetworkX will give us a dictionary that gives every node and its degree, like this:

In [ ]:
nx.degree(random_graph)

...and then we can sort these pairs to find the node that has the highest degree.

In [ ]:
from operator import itemgetter

degrees = nx.degree(random_graph)
sorted(degrees.items(), key=itemgetter(1), reverse=True)[0]

You can also find the shortest path between two nodes:

In [ ]:
nx.shortest_path(random_graph, 2, 15)

...but be careful! If you don't know whether there actually is a path between the nodes, you might get an error.

In [ ]:
nx.shortest_path(random_graph, 1, 17)

So it is usually a good idea to check that you actually got a path. Here is a function that you can write, that looks for the NetworkXNoPath error and handles it if necessary, by printing out a message.

In [ ]:
def find_the_way(graph, here, there):
    try:
        print("The route you want is ", nx.shortest_path(graph, here, there))
    except nx.NetworkXNoPath:
        print("You can't get there from here.")
        
find_the_way(random_graph, 10, 9)

As it happens, there is another way to do this - NetworkX also has a method called `has_path`. It works like this.

In [ ]:
def find_the_way(graph, here, there):
    if nx.has_path(graph, here, there):
        print("The route you want is ", nx.shortest_path(graph, here, there))
    else:
        print("You can't get there from here.")
        
find_the_way(random_graph, 1, 9)

These things change a little bit if you use a directed graph - in the first place, you have to distinguish between "in-degree" (the number of edges pointing *to* a node) vs. "out-degree" (the number of edgs pointing *from* a node.) Think of the difference between someone who follows a lot of people on Twitter vs. someone who has a lot of followers.

We can make a directed graph in the same way we made an undirected one, with an extra parameter to say that it should be directed.

In [ ]:
random_directed_graph = nx.erdos_renyi_graph(20, 0.08, directed=True)
%dotstr -f svg nx.nx_agraph.to_agraph(random_directed_graph).string()

In [ ]:
find_the_way(random_directed_graph, 0, 1)
find_the_way(random_directed_graph, 1, 0)

What are some situations where you might want to use an undirected graph? How about a directed graph?

Social networks and their analysis
-----

For the rest of this exercise we will use some real-life data, taken from the [Prosopography of the Byzantine World](http://db.pbw.kcl.ac.uk/jsp/index.jsp). I have extracted from that database the information on how people were related to each other, counting only sets with at least five people related to each other (that is, subgraphs with a minimum node size of five). The information is in the CSV file called `byzkinship.csv`. So we need to parse it into a graph.

In [ ]:
import csv

kinship = nx.DiGraph()
with open('../lessondata/byzkinship.csv', newline='') as kinfile:
    kinreader = csv.DictReader(kinfile)
    for row in kinreader:
        kinship.add_edge(row['Person'], row['Relative'], kinship=row['Relationship'])
        
len(kinship.nodes())

So let's find the subgraphs!

In [ ]:
for subgraph in nx.connected_components(kinship):
    print(len(subgraph))

Oh dear, we have a problem: we can only find subgraphs in NetworkX with undirected graphs. This does make sense - if you can't get from node 1 to node 2, even when you can get from node 2 to node 1, can you really say they are connected? It's something that you can argue about. We can get around this though, if we do want to ignore the direction of the edges, by telling NetworkX to treat our graph temporarily as an undirected one.

In [ ]:
for subgraph in nx.connected_components(kinship.to_undirected()):
    print(len(subgraph))

We can make a visualization of this graph. HOWEVER, if we use Graphviz the graph will be unthinkably huge, so we probably don't want to do that. Instead we can use the matplotlib library, which is included with Anaconda and which networkx also supports. The graph will be too dense to be deeply meaningful, but you will be able to see its shape.

In [ ]:
import matplotlib.pyplot as plt

nx.draw(kinship, node_size=2)
plt.show()

There are a few measures of relevance in social network analysis. One of them is node degree - that is, who has the highest number of connections? Who, in this graph, is related to the most other people? Let's find out.

In [ ]:
degrees = nx.degree(kinship)
sorted(degrees.items(), key=itemgetter(1), reverse=True)[0:10]

This can also be expressed as "degree centrality" - quite simply, the degree divided by the number of nodes in the graph. This provides an easy way to compare the centrality of nodes from two different graphs even if they are not the same size.

In [ ]:
degree_centrality = nx.degree_centrality(kinship)
sorted(degree_centrality.items(), key=itemgetter(1), reverse=True)[0:10]

You see that it is the same list, just with different numbers.

It also tends to happen in social networks that, even when all the nodes are connected to each other, you can tell that there are subgroups. Think of your friends from school vs. your friends from your department - each subset knows most of the others in that subset, but you may be one of the very few who belongs to both sets. That puts you *between* the groups, and *central* to connecting them. In social networking terms this is *betweenness centrality* - which people tend to sit between distinct clusters of people and so connect them?

In [ ]:
central = nx.betweenness_centrality(kinship)
sorted(central.items(), key=itemgetter(1), reverse=True)[0:10]

Finally, we can look for these distinct clusters within a graph. To do this, let's find the one big part of the graph we saw before, with 698 connected people.

In [ ]:
extended_family = None
for subgraph in nx.connected_component_subgraphs(kinship.to_undirected()):
    if len(subgraph) > 600:
        extended_family = subgraph

nx.draw(extended_family, node_size=2)
plt.show()

We can see some distinct clusters in there; can NetworkX find them? This is a feature known as 'community detection', and in order to use it you need to install an extra module:

    $ pip install python-louvain
    
Once you've done that, you will import the module under the name `community`. Confusing, I know.

In [ ]:
import community

partition = community.best_partition(extended_family)
partition

So let's visualize that. We will need to draw each group of nodes separately, with a different color for each group number.

In [ ]:
subgroups = {}
for person, gnum in partition.items():
    if gnum in subgroups:
        subgroups[gnum].append(person)
    else:
        subgroups[gnum] = [person]
subgroups

In [ ]:
pos = nx.spring_layout(extended_family)
colors = ['#767e1c', '#5ff17b', '#a67e3a', '#0a8c2a', '#a5a319', '#13b7e1', '#7431ae', 
          '#575b98', '#388834', '#912686', '#1eaa4c', '#6bae56', '#6d0647', '#e71cbd', '#000000', 
          '#7d649a', '#cad51f', '#2acc22', '#912f54', '#8b346d', '#faf73c', '#713206', '#ffffff']
for gnum in subgroups.keys():
    color = "C%d" % gnum
    nx.draw_networkx_nodes(extended_family, pos, nodelist=subgroups[gnum], node_color=colors[gnum], node_size=30)
nx.draw_networkx_edges(extended_family, pos)
plt.show()

And so we're done! And as this is digital humanities you may now ask yourself: what exactly does it imply, to have distinct clusters within a group of kin...? What do you suppose we have actually found, if anything?